In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print(tf.__version__)
print(keras.__version__)

1.14.0
2.2.4-tf


## GoogleDriveをマウントする。

In [2]:
import os
if os.name == 'nt':
    print('OS is Windows: PASS mount google drive')
    g_dir_work = '/Users/mhomm/GoogleDrive/colab/'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    g_dir_work = '/content/drive/My Drive/colab/'

# check mount point
print('\n<< Display work dir >>')
for file in os.listdir(g_dir_work):
    print( 'file/dir : ', file)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive

<< Display work dir >>
file/dir :  mylib
file/dir :  test.txt
file/dir :  data
file/dir :  template.ipynb
file/dir :  .ipynb_checkpoints
file/dir :  lt6
file/dir :  lt7
file/dir :  models
file/dir :  mnist_cams


## local Libraryパスを通す。

In [0]:
import sys
if os.name == 'nt':# windows
    lib_path='/Users/mhomm/GoogleDrive/colab/mylib'
else:
    lib_path='/content/drive/My Drive/colab/mylib/'
sys.path.append(lib_path)

In [4]:
# check lib path
import myfunc
myfunc.test()

myfunc.test


以上、テンプレート

---

### データの前処理

In [5]:
import datasets
import tensorflow.keras as keras
import numpy as np

data = datasets.Dataset_mnist()
data.load()
data.x_train = data.x_train.reshape(data.x_train.shape[0], data.x_train.shape[1], data.x_train.shape[2], 1) / 255.
data.x_val   = data.x_val.reshape(data.x_val.shape[0], data.x_val.shape[1], data.x_val.shape[2], 1)   / 255.

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step


In [6]:
data.dbg_check()

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


### 訓練用モデル構築

In [7]:
import models
model = models.Model_mnist_pix2pix(b_training=True)
model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adam())

W0814 13:56:06.696420 139688833701760 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model_mnist_pix2pix"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
imgs (InputLayer)            [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        51264     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        51232     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 1)         801       
Total params: 104,129
Trainable params: 104,129
Non-trainable params: 0
_________________________________________________________________


### モデルの最適化

In [0]:
batch_size = 1000
epochs = 10

In [9]:
print(data.y_train.shape)
print(data.x_train.shape)
print(data.y_val.shape)
print(data.x_val.shape)

(60000,)
(60000, 28, 28, 1)
(10000,)
(10000, 28, 28, 1)


In [10]:
history = model.fit(data.x_train, data.x_train,  # 入出力同じ画像
                    batch_size=batch_size,
                    epochs=epochs,     # エポック数の指定
                    verbose=1,         # ログ出力の指定. 0だとログが出ない
                    validation_data=(data.x_val, data.x_val))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 13s 225us/sample - loss: 0.0361 - val_loss: 0.0031
Epoch 2/10
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0021 - val_loss: 0.0015
Epoch 3/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.0012 - val_loss: 9.6961e-04
Epoch 4/10
60000/60000 [==============================] - 6s 95us/sample - loss: 0.0011 - val_loss: 8.2571e-04
Epoch 5/10
60000/60000 [==============================] - 6s 95us/sample - loss: 7.4639e-04 - val_loss: 6.6903e-04
Epoch 6/10
60000/60000 [==============================] - 6s 96us/sample - loss: 6.2251e-04 - val_loss: 5.6919e-04
Epoch 7/10
60000/60000 [==============================] - 6s 97us/sample - loss: 5.4770e-04 - val_loss: 6.9409e-04
Epoch 8/10
60000/60000 [==============================] - 6s 97us/sample - loss: 4.9674e-04 - val_loss: 4.5029e-04
Epoch 9/10
60000/60000 [============================

### モデルの保存

In [11]:
import datetime
dt_now = datetime.datetime.now()

save_dir = '{0}/models/mnist_pix2pix/{1}/'.format(g_dir_work, dt_now.strftime('%Y%m%d_%H%M%S') )
print(save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model.save( save_dir + '/model.h5' )

/content/drive/My Drive/colab//models/mnist_pix2pix/20190814_135712/


In [0]:
del model

---

### 推定モデルの読み込み

In [13]:
model_pred = models.Model_mnist_pix2pix(b_training=False, model_path=save_dir + '/model.h5')

Model: "model_mnist_pix2pix_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
imgs (InputLayer)            [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        51264     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        51232     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 1)         801       
Total params: 104,129
Trainable params: 104,129
Non-trainable params: 0
_________________________________________________________________


In [0]:
from PIL import Image
def save_img(img, file):
    img[ img <   0 ] = 0
    img[ img > 255 ] = 255
    if os.path.exists(os.path.dirname(file)) is False:
        os.makedirs( os.path.dirname(file) )
    Image.fromarray(img.astype(np.uint8)).save(file)

In [0]:
for i in range(20):
    x = data.x_val[i]
    result = model_pred.predict( x.reshape(1,28,28,1) )

    shape = (x.shape[0], x.shape[1])
    save_img( x.reshape(shape)*255, g_dir_work + 'mnist_pix2pix/{0:03d}_input.png'.format(i) )
    save_img( result[0].reshape(shape)*255, g_dir_work + 'mnist_pix2pix/{0:03d}_pred.png'.format(i) )